In [1]:
import subprocess
import time
import os 
import sys 
import io
import functions
import importlib
import inspect

import warnings 
warnings.filterwarnings('ignore')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
importlib.reload(functions)
from functions import *

pd.set_option('display.max_columns', 100)

# Get all functions in the module
functions_list = [f[0] for f in inspect.getmembers(functions, inspect.isfunction)]
print(functions_list)

['accuracy_score', 'add_missing_ons_vars', 'assign_elections', 'convert_age', 'create_dummies', 'create_factors', 'drop_dups', 'get_BES_data', 'get_data', 'impute_loc_codes', 'index_by_constituency', 'make_scorer', 'mean_squared_error', 'merge_results', 'read_dem_data', 'report_perf', 'train_test_split', 'xgboost_model', 'yougov_polls']


In [4]:
pd.read_excel('data/yougov_voting_intention.xlsx', sheet_name='Male').head(5)

,"If there were a general election held tomorrow, which party would you vote for?",2020-01-26,2020-02-02,2020-02-10,2020-04-02,2020-04-17,2020-04-24,2020-05-06,2020-05-19,2020-05-26,2020-05-30,2020-06-12,2020-06-28,2020-07-09,2020-07-23,2020-07-31,2020-08-05,2020-08-12,2020-08-19,2020-08-25,2020-09-04,2020-09-09,2020-09-17,2020-09-24,2020-09-30,2020-10-07,2020-10-15,2020-10-22,2020-10-29,2020-11-05,2020-11-12,2020-11-18,2020-11-27,2020-12-03,2020-12-09,2020-12-16,2020-12-22,2021-01-05,2021-01-14,2021-01-22,2021-01-27,2021-02-03,2021-02-10,2021-02-18,2021-02-26,2021-03-04,2021-03-10,2021-03-19,2021-03-26,2021-04-01,...,2023-08-03,2023-08-11,2023-08-18,2023-08-23,2023-08-31,2023-09-08,2023-09-14,2023-09-20,2023-09-22,2023-09-27,2023-10-05,2023-10-12,2023-10-18,2023-10-25,2023-11-01,2023-11-08,2023-11-15,2023-11-23,2023-11-30,2023-12-07,2023-12-13,2023-12-20,2024-01-03,2024-01-11,2024-01-17,2024-01-24,2024-01-31,2024-02-08,2024-02-15,2024-02-21,2024-02-29,2024-03-07,2024-03-13,2024-03-20,2024-03-27,2024-04-03,2024-04-11,2024-04-17,2024-04-24,2024-05-01,2024-05-08,2024-05-16,2024-05-22,2024-05-24,2024-05-28,2024-05-30,2024-06-04,2024-06-06,2024-06-11,2024-06-13
0,Con,0.52,0.53,0.49,0.54,0.51,0.49,0.50,0.51,0.44,0.46,0.47,0.48,0.46,0.44,0.43,0.40,0.44,0.39,0.44,0.44,0.45,0.41,0.40,0.43,0.42,0.40,0.43,0.41,0.36,0.40,0.39,0.39,0.41,0.39,0.40,0.43,0.40,0.40,0.40,0.39,0.40,0.41,0.40,0.41,0.44,0.43,0.45,0.44,0.42,...,0.26,0.24,0.25,0.21,0.28,0.22,0.25,0.26,0.28,0.24,0.23,0.24,0.24,0.23,0.21,0.25,0.21,0.26,0.22,0.21,0.21,0.23,0.24,0.20,0.20,0.20,0.22,0.20,0.23,0.21,0.18,0.19,0.20,0.17,0.21,0.19,0.19,0.21,0.20,0.16,0.18,0.19,0.22,0.20,0.17,0.21,0.19,0.19,0.17,0.17
1,Lab,0.27,0.26,0.27,0.27,0.30,0.31,0.30,0.31,0.37,0.32,0.35,0.35,0.35,0.32,0.34,0.37,0.34,0.35,0.32,0.35,0.34,0.39,0.36,0.34,0.38,0.36,0.36,0.33,0.34,0.36,0.35,0.35,0.33,0.34,0.36,0.36,0.35,0.37,0.35,0.39,0.36,0.33,0.34,0.33,0.33,0.31,0.33,0.29,0.31,...,0.44,0.42,0.42,0.43,0.41,0.45,0.40,0.40,0.40,0.45,0.43,0.45,0.45,0.45,0.44,0.43,0.44,0.41,0.41,0.42,0.42,0.42,0.43,0.45,0.45,0.46,0.41,0.45,0.43,0.42,0.43,0.44,0.39,0.41,0.37,0.42,0.44,0.40,0.43,0.43,0.45,0.45,0.45,0.43,0.46,0.43,0.38,0.38,0.37,0.34
2,Lib Dem,0.10,0.08,0.09,0.08,0.07,0.06,0.07,0.06,0.08,0.07,0.07,0.05,0.08,0.08,0.07,0.08,0.05,0.07,0.08,0.08,0.07,0.06,0.08,0.05,0.05,0.06,0.06,0.06,0.08,0.06,0.08,0.05,0.06,0.09,0.05,0.04,0.06,0.06,0.05,0.06,0.07,0.07,0.08,0.06,0.07,0.07,0.04,0.08,0.07,...,0.11,0.11,0.11,0.09,0.09,0.11,0.11,0.12,0.10,0.11,0.10,0.09,0.10,0.10,0.10,0.09,0.10,0.09,0.11,0.11,0.10,0.09,0.11,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.08,0.10,0.10,0.10,0.11,0.07,0.08,0.08,0.08,0.09,0.10,0.10,0.09,0.10,0.10,0.10,0.10,0.11,0.14,0.14
3,SNP,0.05,0.05,0.05,0.04,0.05,0.06,0.03,0.05,0.04,0.05,0.04,0.04,0.04,0.06,0.04,0.05,0.07,0.06,0.06,0.05,0.05,0.03,0.04,0.06,0.05,0.04,0.05,0.06,0.05,0.04,0.06,0.06,0.05,0.05,0.05,0.04,0.05,0.05,0.05,0.04,0.05,0.05,0.05,0.05,0.04,0.04,0.05,0.04,0.05,...,0.03,0.03,0.03,0.04,0.03,0.04,0.03,0.04,0.04,0.03,0.04,0.04,0.04,0.04,0.02,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.04,0.04,0.03,0.04,0.03,0.03,0.04,0.04,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.04,0.02,0.03,0.04,0.04,0.03,0.02,0.03,0.02,0.03,0.03,0.02,0.03
4,Plaid Cymru,0.00,0.00,0.00,0.01,0.01,0.01,0.00,0.01,0.01,0.00,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.01,0.02,0.00,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.00,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.01,0.02,0.01,0.01,0.01,...,0.01,0.00,0.01,0.01,0.01,0.01,0.01,0.00,0.00,0.01,0.01,0.01,0.00,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.00,0.01,0.01,0.01,0.01,0.01,0.00,0.00,0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.00,0.01,0.01,0.01,0.01,0.01,0.00,0.00,0.01,0.00,0.01,0.02,0.01,0.01


In [250]:
cols_to_agg = ['Male','Female', 'ABC1', 'C2DE', '18-24', '25-49', '50-64', '65+', 'Remain', 'Leave', 'All adults']

polls = pd.DataFrame()
for col in cols_to_agg:
    
    if col != 'All adults':
        x = yougov_polls(col)
        polls = pd.concat([polls, x], axis=0)
    else:
        x = yougov_polls(col)[['Unweighted base', 'Base', 'Con', 'Lab','Lib Dem', 'SNP', 'Plaid Cymru',
                               'Reform UK','Green', 'Other']]
        
        x.rename(columns={'Unweighted base': 'All adults Unweighted base', 'Base': 'All adults Base', 'Con': 'Con_all', 'Lab': 'Lab_all', 'Lib Dem': 'Lib_dem_all', 'SNP': 'SNP_all', 'Plaid Cymru': 'PC_all', 'Reform UK': 'Reform_all', 'Green': 'Green_all', 'Other': 'Other_all' }, inplace=True)
        polls = pd.concat([polls, x], axis=1)
    


polls['Proportion_unweighted'] = polls['Unweighted base'] / polls['All adults Unweighted base']
polls['Proportion_weighted'] = polls['Base'] / polls['All adults Base']
polls.demographic = polls.demographic.str.replace('-', '_')

polls

"If there were a general election held tomorrow, which party would you vote for?",date,Con,Lab,Lib Dem,SNP,Plaid Cymru,Reform UK,Green,Other,Unweighted base,Base,demographic,All adults Unweighted base,All adults Base,Con_all,Lab_all,Lib_dem_all,SNP_all,PC_all,Reform_all,Green_all,Other_all,Proportion_unweighted,Proportion_weighted
0,2020-01-26,0.52,0.27,0.1,0.05,0.0,0.02,0.03,0.0,542.0,548.0,Male,1255.0,1096.0,0.49,0.29,0.1,0.05,0.01,0.02,0.04,0.0,0.431873,0.5
1,2020-02-02,0.53,0.26,0.08,0.05,0.0,0.03,0.05,0.0,540.0,539.0,Male,1212.0,1057.0,0.49,0.3,0.08,0.04,0.01,0.02,0.05,0.01,0.445545,0.509934
2,2020-02-10,0.49,0.27,0.09,0.05,0.0,0.03,0.05,0.01,603.0,570.0,Male,1301.0,1157.0,0.48,0.28,0.1,0.04,0.01,0.02,0.06,0.01,0.46349,0.492653
3,2020-04-02,0.54,0.27,0.08,0.04,0.01,0.02,0.04,0.0,555.0,546.0,Male,1242.0,1077.0,0.52,0.28,0.08,0.05,0.01,0.01,0.05,0.01,0.44686,0.506964
4,2020-04-17,0.51,0.3,0.07,0.05,0.01,0.01,0.03,0.02,727.0,673.0,Male,1566.0,1353.0,0.53,0.32,0.05,0.04,0.01,0.01,0.03,0.01,0.46424,0.497413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,2024-05-22,0.35,0.26,0.06,0.01,0.0,0.25,0.05,0.02,548.0,512.0,Leave,1465.0,1330.0,0.21,0.46,0.09,0.03,0.0,0.12,0.07,0.01,0.374061,0.384962
216,2024-05-24,0.37,0.24,0.05,0.01,0.01,0.29,0.02,0.02,554.0,525.0,Leave,1534.0,1403.0,0.22,0.44,0.09,0.03,0.01,0.14,0.06,0.01,0.361147,0.374198
217,2024-05-28,0.34,0.29,0.05,0.01,0.01,0.25,0.04,0.02,595.0,553.0,Leave,1570.0,1405.0,0.2,0.47,0.09,0.03,0.01,0.12,0.07,0.02,0.378981,0.393594
218,2024-05-30,0.38,0.23,0.04,0.01,0.01,0.3,0.02,0.01,586.0,517.0,Leave,1485.0,1329.0,0.21,0.46,0.08,0.02,0.01,0.15,0.06,0.01,0.394613,0.389014


In [6]:
# create constituency level data using census data 
cd = pd.read_csv('data/census_data.csv')

cols_to_index = ['Highest level of qualification (7 categories)', 'Sex (2 categories)', 'Household deprivation (6 categories)', 'Age (6 categories)']


con_df = pd.DataFrame()
for col in cols_to_index:
    z = index_by_constituency(cd, col, col + '_comp')
    con_df = pd.concat([con_df, z], axis=1)
    
    
# drop the multi-level columns
con_df.columns = [' '.join(col).strip() for col in con_df.columns.values]


abc1 = ['Highest level of qualification (7 categories)_comp Level 4 qualifications or above: degree (BA, BSc), higher degree (MA, PhD, PGCE), NVQ level 4 to 5, HNC, HND, RSA Higher Diploma, BTEC Higher level, professional qualifications (for example, teaching, nursing, accountancy)']

cde2 = ['Highest level of qualification (7 categories)_comp Does not apply', 
        'Highest level of qualification (7 categories)_comp Level 1 and entry level qualifications: 1 to 4 GCSEs grade A* to C, Any GCSEs at other grades, O levels or CSEs (any grades), 1 AS level, NVQ level 1, Foundation GNVQ, Basic or Essential Skills',
        'Highest level of qualification (7 categories)_comp Level 2 qualifications: 5 or more GCSEs (A* to C or 9 to 4), O levels (passes), CSEs (grade 1), School Certification, 1 A level, 2 to 3 AS levels, VCEs, Intermediate or Higher Diploma, Welsh Baccalaureate Intermediate Diploma, NVQ level 2, Intermediate GNVQ, City and Guilds Craft, BTEC First or General Diploma, RSA Diploma',
       'Highest level of qualification (7 categories)_comp Level 3 qualifications: 2 or more A levels or VCEs, 4 or more AS levels, Higher School Certificate, Progression or Advanced Diploma, Welsh Baccalaureate Advance Diploma, NVQ level 3; Advanced GNVQ, City and Guilds Advanced Craft, ONC, OND, BTEC National, RSA Advanced Diploma',
       'Highest level of qualification (7 categories)_comp No qualifications',
       'Highest level of qualification (7 categories)_comp Other: apprenticeships, vocational or work-related qualifications, other qualifications achieved in England or Wales, qualifications achieved outside England or Wales (equivalent not stated or unknown)']


# create a new column for the percentage of people with a degree
con_df['abc1'] = con_df[abc1].sum(axis=1)
con_df['cde2'] = con_df[cde2].sum(axis=1)




In [7]:
con_df

,Westminster Parliamentary constituencies Code,Observation Does not apply,"Observation Level 1 and entry level qualifications: 1 to 4 GCSEs grade A* to C, Any GCSEs at other grades, O levels or CSEs (any grades), 1 AS level, NVQ level 1, Foundation GNVQ, Basic or Essential Skills","Observation Level 2 qualifications: 5 or more GCSEs (A* to C or 9 to 4), O levels (passes), CSEs (grade 1), School Certification, 1 A level, 2 to 3 AS levels, VCEs, Intermediate or Higher Diploma, Welsh Baccalaureate Intermediate Diploma, NVQ level 2, Intermediate GNVQ, City and Guilds Craft, BTEC First or General Diploma, RSA Diploma","Observation Level 3 qualifications: 2 or more A levels or VCEs, 4 or more AS levels, Higher School Certificate, Progression or Advanced Diploma, Welsh Baccalaureate Advance Diploma, NVQ level 3; Advanced GNVQ, City and Guilds Advanced Craft, ONC, OND, BTEC National, RSA Advanced Diploma","Observation Level 4 qualifications or above: degree (BA, BSc), higher degree (MA, PhD, PGCE), NVQ level 4 to 5, HNC, HND, RSA Higher Diploma, BTEC Higher level, professional qualifications (for example, teaching, nursing, accountancy)",Observation No qualifications,"Observation Other: apprenticeships, vocational or work-related qualifications, other qualifications achieved in England or Wales, qualifications achieved outside England or Wales (equivalent not stated or unknown)",Highest level of qualification (7 categories)_comp Does not apply,"Highest level of qualification (7 categories)_comp Level 1 and entry level qualifications: 1 to 4 GCSEs grade A* to C, Any GCSEs at other grades, O levels or CSEs (any grades), 1 AS level, NVQ level 1, Foundation GNVQ, Basic or Essential Skills","Highest level of qualification (7 categories)_comp Level 2 qualifications: 5 or more GCSEs (A* to C or 9 to 4), O levels (passes), CSEs (grade 1), School Certification, 1 A level, 2 to 3 AS levels, VCEs, Intermediate or Higher Diploma, Welsh Baccalaureate Intermediate Diploma, NVQ level 2, Intermediate GNVQ, City and Guilds Craft, BTEC First or General Diploma, RSA Diploma","Highest level of qualification (7 categories)_comp Level 3 qualifications: 2 or more A levels or VCEs, 4 or more AS levels, Higher School Certificate, Progression or Advanced Diploma, Welsh Baccalaureate Advance Diploma, NVQ level 3; Advanced GNVQ, City and Guilds Advanced Craft, ONC, OND, BTEC National, RSA Advanced Diploma","Highest level of qualification (7 categories)_comp Level 4 qualifications or above: degree (BA, BSc), higher degree (MA, PhD, PGCE), NVQ level 4 to 5, HNC, HND, RSA Higher Diploma, BTEC Higher level, professional qualifications (for example, teaching, nursing, accountancy)",Highest level of qualification (7 categories)_comp No qualifications,"Highest level of qualification (7 categories)_comp Other: apprenticeships, vocational or work-related qualifications, other qualifications achieved in England or Wales, qualifications achieved outside England or Wales (equivalent not stated or unknown)",Westminster Parliamentary constituencies Code,Observation Female,Observation Male,Sex (2 categories)_comp Female,Sex (2 categories)_comp Male,Westminster Parliamentary constituencies Code,Observation Does not apply,Observation Household is deprived in four dimensions,Observation Household is deprived in one dimension,Observation Household is deprived in three dimensions,Observation Household is deprived in two dimensions,Observation Household is not deprived in any dimension,Household deprivation (6 categories)_comp Does not apply,Household deprivation (6 categories)_comp Household is deprived in four dimensions,Household deprivation (6 categories)_comp Household is deprived in one dimension,Household deprivation (6 categories)_comp Household is deprived in three dimensions,Household deprivation (6 categories)_comp Household is deprived in two dimensions,Household deprivation (6 categories)_comp Household is not deprived in any dimension,Westminster Parliamentary constitu

In [8]:

       
con_df.rename(columns={'Age (6 categories)_comp Aged 15 years and under': 'age_15',
         'Age (6 categories)_comp Aged 16 to 24 years': '18_24',
         'Age (6 categories)_comp Aged 25 to 34 years': '25_34',
         'Age (6 categories)_comp Aged 35 to 49 years': '35_49',
         'Age (6 categories)_comp Aged 50 to 64 years': '50_64',
         'Age (6 categories)_comp Aged 65 years and over': '65+',
         'Sex (2 categories)_comp Female': 'Female', 
         'Sex (2 categories)_comp Male': 'Male',
         'abc1': 'ABC1', 
         'cde2': 'C2DE',
         'Westminster Parliamentary constituencies Code': 'pcon_code'}, inplace=True)

con_df['25_49'] = con_df['25_34'] + con_df['35_49']



In [271]:
cdf = con_df[['pcon_code', '18_24', '25_49', '50_64', '65+', 'ABC1', 'C2DE', 'Male', 'Female']]
cdf = cdf.loc[:,~cdf.columns.duplicated()]
cdf

,pcon_code,18_24,25_49,50_64,65+,ABC1,C2DE,Male,Female
0,E14000530,0.092804,0.371730,0.189605,0.155285,0.244321,0.755679,0.496232,0.503768
1,E14000531,0.089205,0.291424,0.215438,0.225226,0.216603,0.783397,0.487946,0.512054
2,E14000532,0.082755,0.321373,0.195694,0.185915,0.384103,0.615897,0.486705,0.513295
3,E14000533,0.087399,0.306054,0.220229,0.215428,0.203068,0.796932,0.493022,0.506978
4,E14000534,0.076850,0.253693,0.230765,0.274600,0.308221,0.691779,0.484078,0.515922
...,...,...,...,...,...,...,...,...,...
565,W07000076,0.093945,0.317187,0.203083,0.195417,0.220297,0.779703,0.485468,0.514532
566,W07000077,0.094473,0.312245,0.209240,0.203453,0.203534,0.796466,0.492780,0.507220
567,W07000078,0.085795,0.302420,0.209650,0.214513,0.286685,0.713315,0.481311,0.518689
568,W07000079,0.099427,0.347714,0.188309,0.156617,0.317640,0.682360,0.481760,0.518240


In [272]:
brexit = pd.read_csv('data/ge_19_hp_all.csv')

brexit_leave_dict = dict(zip(brexit['ons_const_id'], brexit['leave_hanretty']))
brexit_remain_dict = dict(zip(brexit['ons_const_id'], brexit['remain_hanretty']))

In [273]:
cdf['Leave'] = cdf.pcon_code.map(brexit_leave_dict)
cdf['Remain'] = cdf.pcon_code.map(brexit_remain_dict)

In [274]:
polls_wide = polls.pivot(index='date', columns='demographic', values=['Proportion_unweighted', 'Proportion_weighted', 'Con', 'Lab', 'Lib Dem', 'Green', 'SNP', 'Plaid Cymru', 'Other']).reset_index()
polls_wide

date Proportion_unweighted                                \
demographic                             18_24     25_49     50_64       65+   
0            2020-01-26              0.077291  0.384861  0.255777  0.282072   
1            2020-02-02              0.066007  0.387789    0.2467  0.299505   
2            2020-02-10              0.088394  0.400461  0.262875  0.248271   
3            2020-04-02              0.080515   0.39533  0.252818  0.271337   
4            2020-04-17               0.07599  0.388889  0.254789  0.280332   
..                  ...                   ...       ...       ...       ...   
215          2024-05-22              0.080546  0.408191  0.254608  0.256655   
216          2024-05-24              0.080834  0.404172  0.259452  0.255541   
217          2024-05-28              0.070064  0.393631  0.266242  0.270064   
218          2024-05-30              0.060606  0.379798  0.264646  0.294949   
219          2024-06-04              0.053407  0.381215  0.267649  0.297729   

                                                                         \
demographic      ABC1      C2DE    Female     Leave      Male    Remain   
0            0.622311  0.377689  0.568127  0.433466  0.431873  0.466932   
1            0.627888  0.372112  0.554455  0.420792  0.445545  0.476073   
2            0.605688  0.394312   0.53651  0.441968   0.46349  0.448885   
3            0.608696  0.391304   0.55314  0.455717   0.44686  0.429147   
4            0.621328  0.378672   0.53576  0.441252   0.46424  0.452107   
..                ...       ...       ...       ...       ...       ...   
215          0.620478  0.379522  0.497611  0.374061  0.502389  0.449147   
216          0.638201  0.361799  0.507823  0.361147  0.492177  0.447849   
217          0.625478  0.374522  0.530573  0.378981  0.469427  0.464331   
218          0.616835  0.383165  0.506397  0.394613  0.493603  0.447138   
219          0.636587  0.363413  0.504604  0.400246  0.495396  0.442603   

            Proportion_weighted                                          \
demographic               18_24     25_49     50_64       65+      ABC1   
0                      0.086679  0.381387  0.255474   0.27646  0.613139   
1                      0.102176  0.349101   0.25071  0.298959  0.611164   
2                      0.091616  0.371651  0.266206  0.270527   0.60847   
3                      0.090994   0.38533  0.262767   0.26091  0.611885   
4                      0.091648   0.37694   0.27051  0.261641  0.615669   
..                          ...       ...       ...       ...       ...   
215                    0.082707  0.398496  0.267669  0.251128  0.610526   
216                     0.09551  0.404134  0.256593  0.243763  0.595866   
217                    0.079715  0.409253  0.251246  0.259786  0.600712   
218                    0.086531  0.388262  0.255079  0.270128  0.606471   
219                     0.09822  0.406998  0.246777  0.248005  0.597913   

                                                                Con        \
demographic      C2DE    Female     Leave      Male    Remain 18_24 25_49   
0            0.386861       0.5   0.45073       0.5  0.414234  0.17   0.4   
1            0.389782  0.490066  0.447493  0.509934  0.425733  0.22  0.35   
2             0.39153  0.507347  0.439931  0.492653  0.420052  0.19  0.32   
3            0.388115  0.493036   0.45311  0.506964  0.399257  0.25  0.41   
4            0.384331  0.502587  0.447894  0.497413  0.423503  0.28  0.39   
..                ...       ...       ...       ...       ...   ...   ...   
215          0.389474  0.491729  0.384962  0.508271  0.430827  0.09  0.14   
216          0.404134  0.482537  0.374198  0.517463  0.416964  0.07  0.13   
217          0.399288  0.486833  0.393594  0.513167  0.430605  0.04  0.09   
218          0.393529   0.48909  0.389014  0.510158  0.425132  0.05  0.09   
219          0.402087  0.491713  0.381215  0.508287  0.405157  0.12  0.11   

                                     

In [275]:
polls.loc[polls.demographic == 'Male'][['Con_all', 'Lab_all', 'Lib_dem_all', 'Green_all', 'SNP_all', 'PC_all', 'Reform_all', 'Other_all']]

"If there were a general election held tomorrow, which party would you vote for?",Con_all,Lab_all,Lib_dem_all,Green_all,SNP_all,PC_all,Reform_all,Other_all
0,0.49,0.29,0.1,0.04,0.05,0.01,0.02,0.0
1,0.49,0.3,0.08,0.05,0.04,0.01,0.02,0.01
2,0.48,0.28,0.1,0.06,0.04,0.01,0.02,0.01
3,0.52,0.28,0.08,0.05,0.05,0.01,0.01,0.01
4,0.53,0.32,0.05,0.03,0.04,0.01,0.01,0.01
...,...,...,...,...,...,...,...,...
215,0.21,0.46,0.09,0.07,0.03,0.0,0.12,0.01
216,0.22,0.44,0.09,0.06,0.03,0.01,0.14,0.01
217,0.2,0.47,0.09,0.07,0.03,0.01,0.12,0.02
218,0.21,0.46,0.08,0.06,0.02,0.01,0.15,0.01


In [284]:
X

,18_24,25_49,50_64,65+,ABC1,C2DE,Female,Leave,Male,Remain,weights
date,,,,,,,,,,,
2020-01-26,0.077291,0.384861,0.255777,0.282072,0.622311,0.377689,0.568127,0.433466,0.431873,0.466932,1.000000
2020-02-02,0.066007,0.387789,0.246700,0.299505,0.627888,0.372112,0.554455,0.420792,0.445545,0.476073,1.001370
2020-02-10,0.088394,0.400461,0.262875,0.248271,0.605688,0.394312,0.536510,0.441968,0.463490,0.448885,1.002740
2020-04-02,0.080515,0.395330,0.252818,0.271337,0.608696,0.391304,0.553140,0.455717,0.446860,0.429147,1.004110
2020-04-17,0.075990,0.388889,0.254789,0.280332,0.621328,0.378672,0.535760,0.441252,0.464240,0.452107,1.005479
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-22,0.080546,0.408191,0.254608,0.256655,0.620478,0.379522,0.497611,0.374061,0.502389,0.449147,1.294521
2024-05-24,0.080834,0.404172,0.259452,0.255541,0.638201,0.361799,0.507823,0.361147,0.492177,0.447849,1.295890
2024-05-28,0.070064,0.393631,0.266242,0.270064,0.625478,0.374522,0.530573,0.378981,0.469427,0.464331,1.297260


In [285]:
X = polls_wide.set_index('date')['Con'].reset_index()
Y1 = polls.loc[polls.demographic == 'Male']['Con_all'].values
Y2 = polls.loc[polls.demographic == 'Male']['Lab_all'].values

In [286]:
X#.columns

demographic,date,18_24,25_49,50_64,65+,ABC1,C2DE,Female,Leave,Male,Remain
0,2020-01-26,0.17,0.4,0.51,0.68,0.47,0.51,0.45,0.76,0.52,0.26
1,2020-02-02,0.22,0.35,0.61,0.65,0.48,0.52,0.46,0.75,0.53,0.28
2,2020-02-10,0.19,0.32,0.53,0.76,0.48,0.49,0.47,0.76,0.49,0.27
3,2020-04-02,0.25,0.41,0.57,0.74,0.53,0.52,0.51,0.81,0.54,0.25
4,2020-04-17,0.28,0.39,0.62,0.71,0.51,0.56,0.54,0.8,0.51,0.28
...,...,...,...,...,...,...,...,...,...,...,...
215,2024-05-22,0.09,0.14,0.21,0.37,0.19,0.25,0.2,0.35,0.22,0.11
216,2024-05-24,0.07,0.13,0.23,0.42,0.2,0.24,0.24,0.37,0.2,0.13
217,2024-05-28,0.04,0.09,0.24,0.36,0.18,0.21,0.22,0.34,0.17,0.11
218,2024-05-30,0.05,0.09,0.21,0.42,0.2,0.22,0.2,0.38,0.21,0.1


In [295]:
X = polls_wide.set_index('date')['Proportion_weighted'].reset_index()
Y1 = polls.loc[polls.demographic == 'Male']['Con_all'].values
Y2 = polls.loc[polls.demographic == 'Male']['Lab_all'].values

cols = ['date', '18_24', '25_49', '50_64', '65+', 'ABC1', 'C2DE', 'Female', 'Leave', 'Male', 'Remain']

X.columns = cols

numeric_cols = [i for i in X.columns if 'date' not in i]

# Assign weights based on date (example: more recent dates have higher weight)
X['weights'] = np.linspace(1, 1.3, num=len(X))  # Adjust weights based on your criteria
X = X.set_index('date')


weights_dict = dict(zip(X.index, X['weights']))
X[numeric_cols] = X[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [296]:
X

,18_24,25_49,50_64,65+,ABC1,C2DE,Female,Leave,Male,Remain,weights
date,,,,,,,,,,,
2020-01-26,0.086679,0.381387,0.255474,0.276460,0.613139,0.386861,0.500000,0.450730,0.500000,0.414234,1.000000
2020-02-02,0.102176,0.349101,0.250710,0.298959,0.611164,0.389782,0.490066,0.447493,0.509934,0.425733,1.001370
2020-02-10,0.091616,0.371651,0.266206,0.270527,0.608470,0.391530,0.507347,0.439931,0.492653,0.420052,1.002740
2020-04-02,0.090994,0.385330,0.262767,0.260910,0.611885,0.388115,0.493036,0.453110,0.506964,0.399257,1.004110
2020-04-17,0.091648,0.376940,0.270510,0.261641,0.615669,0.384331,0.502587,0.447894,0.497413,0.423503,1.005479
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-22,0.082707,0.398496,0.267669,0.251128,0.610526,0.389474,0.491729,0.384962,0.508271,0.430827,1.294521
2024-05-24,0.095510,0.404134,0.256593,0.243763,0.595866,0.404134,0.482537,0.374198,0.517463,0.416964,1.295890
2024-05-28,0.079715,0.409253,0.251246,0.259786,0.600712,0.399288,0.486833,0.393594,0.513167,0.430605,1.297260


In [297]:
X_train, X_test, y_train, y_test = train_test_split(X, Y1, test_size=0.2, random_state=123)


weights = X_train.index.map(weights_dict)

X_train = X_train.drop(columns=['weights'])
X_test = X_test.drop(columns=['weights'])


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)




reg = xgb.XGBRegressor(random_state=0, booster='gbtree', objective='reg:squarederror', tree_method='gpu_hist')

reg.fit(X_train, y_train, sample_weight=weights)
# test accuracy of model
preds = reg.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))


RMSE: 0.040562


In [298]:
cdf['predicted_lab_vote'] = reg.predict(cdf[X.drop(columns=['weights']).columns])

In [299]:
cdf.predicted_lab_vote.describe()

count    570.000000
mean       0.441238
std        0.008921
min        0.389237
25%        0.441449
50%        0.442752
75%        0.443059
max        0.448120
Name: predicted_lab_vote, dtype: float64

In [300]:
cdf

,pcon_code,18_24,25_49,50_64,65+,ABC1,C2DE,Male,Female,Leave,Remain,predicted_lab_vote
0,E14000530,0.092804,0.371730,0.189605,0.155285,0.244321,0.755679,0.496232,0.503768,57.897766,42.102234,0.438678
1,E14000531,0.089205,0.291424,0.215438,0.225226,0.216603,0.783397,0.487946,0.512054,67.796350,32.203650,0.447975
2,E14000532,0.082755,0.321373,0.195694,0.185915,0.384103,0.615897,0.486705,0.513295,38.587800,61.412200,0.442331
3,E14000533,0.087399,0.306054,0.220229,0.215428,0.203068,0.796932,0.493022,0.506978,65.299125,34.700875,0.447975
4,E14000534,0.076850,0.253693,0.230765,0.274600,0.308221,0.691779,0.484078,0.515922,49.701106,50.298894,0.442752
...,...,...,...,...,...,...,...,...,...,...,...,...
565,W07000076,0.093945,0.317187,0.203083,0.195417,0.220297,0.779703,0.485468,0.514532,55.135971,44.864029,0.443130
566,W07000077,0.094473,0.312245,0.209240,0.203453,0.203534,0.796466,0.492780,0.507220,58.939917,41.060083,0.447975
567,W07000078,0.085795,0.302420,0.209650,0.214513,0.286685,0.713315,0.481311,0.518689,52.550670,47.449330,0.442607
568,W07000079,0.099427,0.347714,0.188309,0.156617,0.317640,0.682360,0.481760,0.518240,43.822636,56.177364,0.442471
